In this exercise, you'll apply what you learned in the **Inconsistent data entry** tutorial.

# Setup

The questions below will give you feedback on your work. Run the following cell to set up the feedback system.

In [ ]:
from learntools.core import binder
binder.bind(globals())
from learntools.data_cleaning.ex5 import *
print("Setup Complete")

# Get our environment set up

The first thing we'll need to do is load in the libraries and dataset we'll be using. 

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# read in all our data
suicide_attacks = pd.read_csv("../input/pakistansuicideattacks/PakistanSuicideAttacks Ver 11 (30-November-2017).csv", 
                              encoding='Windows-1252')

# set seed for reproducibility
np.random.seed(0)

Run the next code cell to take a quick look at the first few rows of the data.

In [ ]:
suicide_attacks.head()

# 1) Do some preliminary text pre-processing

Run the next cell without changes to take a look at all the unique values in the "Province" column.

In [ ]:
provinces = suicide_attacks['Province'].unique()

# sort them alphabetically and then take a closer look
provinces.sort()
provinces

Convert the column to lowercase and remove any trailing white spaces.

In [ ]:
# convert to lower case
suicide_attacks['City'] = suicide_attacks['City'].str.lower()
# remove trailing white spaces
suicide_attacks['City'] = suicide_attacks['City'].str.strip()

# Check your answer
q1.check()

# 2) Use fuzzy matching to correct inconsistent data entry

Alright, let's take another look at the city column and see if there's any more data cleaning we need to do.

In [ ]:
# get all the unique values in the 'City' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities

It looks like 'kuram agency' and 'kurram agency' should be the same city. Correct the dataframe so that they are.

In [ ]:
# question 2

# Check your answer
q2.check()

# (Optional) More practice

Do any other columns in this dataframe have inconsistent data entry? If you can find any, try to tidy them up.

You can also try reading in the `PakistanSuicideAttacks Ver 6 (10-October-2017).csv` file from this dataset and tidying up any inconsistent columns in that data file.

# Congratulations!

Congratulations for completing the **Data Cleaning** course on Kaggle Learn!